In [ ]:
import numpy as np
from libsvm.svmutil import svm_problem, svm_parameter, svm_train, svm_save_model
from discrete_wavelet_transform import dwt_2d
from local_binary_pattern import lbp
from feature_fusion import concatenate_lbp_dwt
import time
import os
import matplotlib.pyplot as plt
import cv2

Import Functions from train.py

In [ ]:
from train import get_data, spatial_frequency_feature_fusion, prepare_data, train_model, visualize

Run this for preprocessing

- must include /utils
- example path: "/Users/Danniel/Detection-of-GAN-Generated-Images-using-Spatial-Frequency-Domain-Fusion-Data/utils"

In [ ]:
import sys
# path of the folder "Detection-of-GAN-Generated-Images-using-Spatial-Frequency-Domain-Fusion-Data"
sys.path.append("/Users/Danniel/Detection-of-GAN-Generated-Images-using-Spatial-Frequency-Domain-Fusion-Data/utils")

from preprocessing_save import load_image, save_image

# directory for reading
img_real = "/Users/Danniel/Downloads/Try/Real"
img_gan = "/Users/Danniel/Downloads/Try/GAN"

# directory for saving (must be an empty folder)
save_dir_real = "/Users/Danniel/Downloads/Try/Preprocessed_Real"
save_dir_gan = "/Users/Danniel/Downloads/Try/Preprocessed_GAN"

# store the images
image_real = load_image(img_real)
image_gan = load_image(img_gan)

save_image(image_real, save_dir_real)
save_image(image_gan, save_dir_gan)

Provide directory for Preprocessed Real and GAN-Generated Images

In [ ]:
real_directory = "/Users/Danniel/Downloads/Try/Preprocessed_Real"
gan_directory = "/Users/Danniel/Downloads/Try/Preprocessed_GAN"

"get_data" function will load the preprocessed images from the directory and store it in a list

In [ ]:
# run data preparation
real_data = get_data(real_directory)
gan_data = get_data(gan_directory)

Run to perform the proposed "spatial frequency feature fusion" method to extract meaningful features of an image to classify whether it is GAN-Generated or Real. The feature vector of each image will be stored in "fused_features" list

In [ ]:
# run feature extraction & feature fusion
extracted_real = spatial_frequency_feature_fusion(real_data)
extracted_gan = spatial_frequency_feature_fusion(gan_data)

In [ ]:
# for multi-tasking
import asyncio

async def feature_extraction(images):
    await asyncio.sleep(1)
    fused_features = spatial_frequency_feature_fusion(images)

    return fused_features

async def main():
    extracted_real = asyncio.create_task(feature_extraction(real_data))
    extracted_gan = asyncio.create_task(feature_extraction(gan_data))

    e_real = await extracted_real
    e_gan = await extracted_gan

    return e_real, e_gan
   

real, gan = await main()


For Data Visualization
- provide real and gan parameter

In [ ]:
# visualize
visualize(real, gan)

Data Preparation before feeding to the classifier
- assigning of labels
- flattening of feature vector

In [ ]:
# prepare the data
labels, datasets = prepare_data(real, gan)
print(labels, datasets)

Model Training
- Support Vector Machine
- Linear Kernel

In [ ]:
# train the data
model = train_model(labels, datasets )

Save the model using ".model" extension

In [ ]:
# save the model
model_file = "/Users/Danniel/Downloads/faces_try(preprocessed).model"
svm_save_model(model_file, model)

Model Testing
- provide directory (folder) for testing datasets

In [ ]:
from libsvm.svmutil import svm_problem, svm_train, svm_save_model, svm_load_model, svm_predict
from preprocessing import preprocessing
import os
import numpy as np
from train import spatial_frequency_feature_fusion

# test the model
def predict(directory):
    images = []

    # load the images and store in images list
    for filename in os.listdir(directory):
        image = os.path.join(directory, filename)
        if image is not None:
            images.append(image)
      


    # load the model
    model_file = "/Users/Danniel/Downloads/faces_validate.model"
    loaded_model = svm_load_model(model_file)

    # preprocessing
    preprocessed_img = []
    for i in images:
        preprocessed_img.append(preprocessing(i))   

    # flatten the feature vector
    fused_features = spatial_frequency_feature_fusion(preprocessed_img)
    labels = np.ones((len(fused_features), 1)) 
    true_label = labels.reshape(labels.shape[0])

    feature_vector = []
    for i in fused_features:
        print(i)
        feature_vector.append(i.flatten())

    print(feature_vector)



    # predict the result
    print("\n\n-------------------THE MODEL IS PREDICTING----------------------------\n")
    predicted_labels, _, likelihood = svm_predict(true_label, feature_vector, loaded_model, '-b 1')


    print("------------------------------------------RESULT-----------------------------------\n")
    result = []
    for i in predicted_labels:
        if i == 1.0:
            result.append("Real")
        elif i == 0.0:
            result.append("GAN")

#provide directory for testing dataset
dir = "/Users/Danniel/Downloads/Low Dataset/test_combination"

predict(dir)

# existing_model_file = "/Users/Danniel/Downloads/sample.model"
# existing_model = svm_load_model(existing_model_file)

# # New data
# new_labels = np.array(new_labels)
# new_datasets = np.array(new_datasets)

# # Train or update the model incrementally
# updated_model = train_model(existing_model, new_labels, new_datasets)

# # Save the updated model
# model_file = "/Users/Danniel/Downloads/updated_model.model"
# svm_save_model(model_file, updated_model)

# # Use the updated model for prediction
# image_directory = "/Users/Danniel/Downloads/Low Dataset/test"
# images = [os.path.join(image_directory, filename) for filename in os.listdir(image_directory)]
# predict(updated_model, images)

Testing combination

In [ ]:
from libsvm.svmutil import svm_problem, svm_train, svm_save_model, svm_load_model, svm_predict
from preprocessing import preprocessing
import os
import numpy as np
from train import spatial_frequency_feature_fusion

# test the model
def get_data(directory):
    images = []

    # load the images and store in images list
    for filename in os.listdir(directory):
        image = os.path.join(directory, filename)
        if image is not None:
            images.append(image)

    # preprocessing
    preprocessed_img = []
    for i in images:
        preprocessed_img.append(preprocessing(i))   

    # flatten the feature vector
    fused_features = spatial_frequency_feature_fusion(preprocessed_img)
    labels = np.ones((len(fused_features), 1)) 
    true_label = labels.reshape(labels.shape[0])

    feature_vector = []
    for i in fused_features:
        print(i)
        feature_vector.append(i.flatten())


    return feature_vector


def predict(real, gan):  
    # load the model
    model_file = "/Users/Danniel/Downloads/faces_validate.model"
    loaded_model = svm_load_model(model_file)

    #label real  and gan datasets
    real_label = np.ones((len(real), 1))
    gan_label = np.zeros((len(gan), 1))

    # combine the labels and datasets
    dataset_labels = np.vstack((real_label, gan_label))
    datasets = np.vstack((real, gan))

    feature_vector = [i.flatten() for i in datasets]
    true_label = dataset_labels.reshape(dataset_labels.shape[0])

    # predict the result
    print("\n\n-------------------THE MODEL IS PREDICTING----------------------------\n")
    predicted_labels, _, likelihood = svm_predict(true_label, feature_vector, loaded_model, '-b 1')


    print("------------------------------------------RESULT-----------------------------------\n")
    result = []
    for i in predicted_labels:
        if i == 1.0:
            result.append("Real")
        elif i == 0.0:
            result.append("GAN")

    print(predicted_labels)        


#provide directory for testing dataset
dir_real = "/Users/Danniel/Downloads/Low Dataset/test_real"
dir_gan = "/Users/Danniel/Downloads/Low Dataset/test_gan"



real = get_data(dir_real)
gan = get_data(dir_gan)

predict(real, gan)